In [5]:
# Celda 1
import duckdb
import pandas as pd

# Configuración opcional: Para que Pandas muestre más columnas si la tabla es ancha
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

print("✅ Librerías importadas correctamente")

✅ Librerías importadas correctamente


In [6]:
# Celda 2: Cargar los datos a DuckDB
con = duckdb.connect()

# 1. Cargamos los logs de acceso (Tu tabla principal)
con.sql("""
    CREATE OR REPLACE TABLE access_logs AS 
    SELECT * FROM '../data/logs_access_logs.json'
""")

# 2. Cargamos los errores y servidores (Tablas de apoyo)
con.sql("""
    CREATE OR REPLACE TABLE error_logs AS 
    SELECT * FROM '../data/logs_access_logs.json'
""")

con.sql("""
    CREATE OR REPLACE TABLE servers AS 
    SELECT * FROM '../data/logs_access_logs.json'
""")

# 3. Verificamos
print("📊 Tablas creadas:")
con.sql("SHOW TABLES").show()

IOException: IO Error: Could not read file "../data/logs_access_logs.json" (error in ReadFile(location: 0, nr_bytes: 2250390)): El proveedor de sincronización en la nube no pudo realizar la operación porque la red no está disponible.


In [ ]:
#Query 1: Exploración Inicial

from IPython.display import display #importante para mostrar dataframes en Jupyter

# Vamos a ver las primeras 5 filas para entender qué columnas tenemos
print("Muestra de access_logs:")
display(con.sql("SELECT * FROM access_logs LIMIT 5").df())


print("Cantidad de datos:")
display(con.sql("SELECT COUNT(*) AS total_rows FROM access_logs").df())

print("Periodo de los datos:")
display(con.sql("""
    SELECT FIRST_VALUE(timestamp) OVER (ORDER BY timestamp) AS first_timestamp,
            LAST_VALUE(timestamp) OVER (ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS last_timestamp
    FROM access_logs
    LIMIT 1
""").df())

print("Usuarios únicos:")
display(con.sql("""
    SELECT COUNT(DISTINCT user_id) AS unique_users
    FROM access_logs
""").df())

Muestra de access_logs:


,log_id,timestamp,service_id,server_id,trace_id,span_id,method,endpoint,status_code,response_time_ms,request_size_bytes,response_size_bytes,user_agent,client_ip,user_id
0,1,2024-03-13 05:06:46,8,7,trace-9abe2e3d3055ff05,span-71608dc3,POST,/api/auth/login,404,40,326,57577,Mozilla/5.0,96.123.163.20,<NA>
1,2,2025-07-17 16:26:44,2,12,trace-a73a9c8658488e40,span-56a0a80f,GET,/api/orders,503,22770,39474,501,PostmanRuntime/7.29.0,6.55.171.77,5545
2,3,2024-04-17 16:14:55,10,2,trace-d3a7df6aa5c8c23f,span-650546b0,POST,/api/products,200,180,15453,22187,python-requests/2.28.0,48.246.10.118,<NA>
3,4,2024-09-08 19:06:05,3,4,trace-6646f8891d89bcb5,span-4e81d949,POST,/api/users,200,40,11293,17883,PostmanRuntime/7.29.0,33.70.37.102,<NA>
4,5,2024-01-14 11:16:27,2,12,trace-5f33d7ef9443422a,span-ed32e9bf,GET,/api/auth/login,400,424,39337,31075,Mozilla/5.0,9.180.191.49,8624


Cantidad de datos:


,total_rows
0,5000


Periodo de los datos:


,first_timestamp,last_timestamp
0,2024-01-01 03:01:31,2025-12-22 23:43:23


Usuarios únicos:


,unique_users
0,2846


In [ ]:
# Query 2: Endpoints mas utilizados

display(con.sql("""
    SELECT endpoint, COUNT(endpoint) AS usage_count, 
    FROM access_logs
    GROUP BY endpoint
    ORDER BY usage_count DESC
""").df())

In [ ]:
# Query 3: Errores por endpoint y usuarios afectados

display(con.sql("""
    SELECT 
            endpoint,
            status_code,
            COUNT(status_code) AS total_errors,
            COUNT(DISTINCT user_id) AS unique_users,
    FROM access_logs
    GROUP BY endpoint, status_code
    ORDER BY endpoint, total_errors DESC
""").df())

In [ ]:
# Query 4: Tiempos de respuesta por endpoint

display(con.sql("""
    SELECT
            endpoint,
            AVG(response_time_ms) AS avg_response_time,
            quantile(response_time_ms, 0.95) AS p95_response_time
    FROM access_logs
    GROUP BY endpoint
    ORDER BY avg_response_time DESC
""").df())

In [ ]:
# Query 5: Análisis temporal de las solicitudes

display(con.sql("""
    SELECT
            EXTRACT(HOUR FROM timestamp) AS hour_of_day,
            COUNT(*) AS requests_count,
            AVG(response_time_ms) AS avg_response_time,
            quantile(response_time_ms, 0.95) AS p95_response_time
    FROM access_logs    
    GROUP BY hour_of_day
""").df())

In [ ]:


display(con.sql("""
    SELECT
            method,
            endpoint,
            AVG(response_time_ms) AS avg_response_time,
            RANK() OVER (PARTITION BY method ORDER BY avg_response_time DESC) AS response_time_rank
    FROM access_logs
    GROUP BY method, endpoint
""").df())

,method,endpoint,avg_response_time,response_time_rank
0,DELETE,/health,5122.547170,1
1,DELETE,/api/auth/logout,4971.887097,2
2,DELETE,/api/users,4431.983871,3
3,DELETE,/api/auth/login,4093.758065,4
4,DELETE,/metrics,3794.230769,5
5,DELETE,/api/cart,3197.619048,6
6,DELETE,/api/payments,3187.365079,7
7,DELETE,/api/products,2948.519481,8
8,DELETE,/api/checkout,2934.865672,9
9,DELETE,/api/search,2301.271186,10
